# Leetcode Premium Problems

In [2]:
from typing import List
from typing import Optional
import pandas as pd
import numpy as np

## 1757. Recyclable and Low Fat Products [E]

Table: Products
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| low_fats    | enum    |
| recyclable  | enum    |
+-------------+---------+
product_id is the primary key (column with unique values) for this table.
low_fats is an ENUM (category) of type ('Y', 'N') where 'Y' means this product is low fat and 'N' means it is not.
recyclable is an ENUM (category) of types ('Y', 'N') where 'Y' means this product is recyclable and 'N' means it is not.
```

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

The result format is in the following example.



In [6]:
# Beats 33.89%
def find_products(products: pd.DataFrame) -> pd.DataFrame:
    return products.loc[ (products['low_fats']=='Y') & (products['recyclable']=='Y'), ['product_id'] ]

In [3]:
data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype({'product_id':'int64', 'low_fats':'category', 'recyclable':'category'})

In [9]:
find_products(products)

,product_id


## 2877. Create a DataFrame from List [E]

Write a solution to create a DataFrame from a 2D list called `student_data`. This 2D list contains the IDs and ages of some students.

The DataFrame should have two columns, `student_id` and `age`, and be in the same order as the original 2D list.

The result format is in the following example.

In [12]:
# Beats 5.06%
def createDataframe(student_data: List[List[int]]) -> pd.DataFrame:
    return pd.DataFrame(student_data, columns=['student_id', 'age'])

In [10]:
student_data = [
  [1, 15],
  [2, 11],
  [3, 11],
  [4, 20]
]

In [13]:
createDataframe(student_data)

,student_id,age
0,1,15
1,2,11
2,3,11
3,4,20
